In [ ]:
# Install semantic-link-labs package
!pip install semantic-link-labs
!pip install tabulate

In [ ]:
# Import Libraries
import sempy_labs as labs
import requests
import pandas as pd
import sempy.fabric as fabric
from tabulate import tabulate

Setup Parameters

In [ ]:
# Parameters
workspace_id = 'your_workspace_id'  # Replace with your actual workspace ID
subscription_key = mssparkutils.credentials.getSecret('your_key_vault_address','lint-subscription-key')

## Define Functions

In [ ]:
import requests
import json
from urllib.parse import urlencode

# API Configuration
API_BASE_URL = "https://api.pqlint.com/v1"

class PQLintApiError(Exception):
    pass

def invoke_pqlint_api_request(uri, method='GET', headers=None, body=None, content_type='application/json'):
    """
    Helper function to make HTTP requests with proper error handling.
    """
    headers = headers or {}
    if content_type:
        headers['Content-Type'] = content_type

    try:
        if method.upper() == 'GET':
            response = requests.get(uri, headers=headers)
        elif method.upper() == 'POST':
            if isinstance(body, (dict, list)):
                response = requests.post(uri, headers=headers, data=json.dumps(body))
            else:
                response = requests.post(uri, headers=headers, data=body)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")

        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            error_body = response.json()
            error_message = f"HTTP {response.status_code} {response.reason} - {error_body.get('body', str(error_body))}"
        except Exception:
            error_message = f"HTTP {response.status_code} {response.reason} - {http_err}"
        raise PQLintApiError(error_message)
    except Exception as err:
        raise PQLintApiError(f"PQLint API Error: {str(err)}")

def get_linting_rules():
    """
    Retrieves all linting rules from the PQLint API.
    """
    uri = f"{API_BASE_URL}/lint/rules"
    return invoke_pqlint_api_request(uri, method='GET')

def invoke_code_linting(code, subscription_key, rules=None, severity=None, format='pq'):
    """
    Lints Power Query M code or TMDL code using the PQLint API.
    """
    if not code.strip():
        raise ValueError("Code parameter cannot be null or empty")
    if not subscription_key.strip():
        raise ValueError("SubscriptionKey parameter cannot be null or empty")

    request_body = {
        "code": code
    }

    if rules:
        request_body["rules"] = rules

    options = {}
    if severity:
        options["severity"] = severity
    if format:
        options["format"] = format

    if options:
        request_body["options"] = options

    encoded_key = urlencode({"subscription-key": subscription_key})
    uri = f"{API_BASE_URL}/pq/lint?{encoded_key}"

    return invoke_pqlint_api_request(uri, method='POST', body=request_body)


def lint_result_to_df(lint_result, dataset_name, table_name, partition_name):
    """
    Converts lint_result (JSON/dict/list) to a DataFrame and adds DatasetName, TableName, and PartitionName columns.

    Args:
        lint_result (dict or list): The lint result JSON or list of rule violations.
        dataset_name (str): The dataset name to add.
        table_name (str): The table name to add.
        partition_name (str): The partition name to add.

    Returns:
        pd.DataFrame: Expanded DataFrame with relevant columns including DatasetName, TableName, PartitionName.
    """
    if isinstance(lint_result, list):
        df = pd.json_normalize(
            lint_result,
            sep='_',
            meta=['ID', 'Name', 'Category', 'Description', 'Severity'],
            errors='ignore'
        )
    elif isinstance(lint_result, dict):
        df = pd.json_normalize([lint_result], sep='_')
    else:
        raise ValueError("lint_result must be a dict or list")

    # Optional: Flatten References list to semicolon-separated string
    if 'References' in df.columns:
        df['References'] = df['References'].apply(lambda refs: '; '.join(
            f"{r.get('Description', '')} ({r.get('Link', '')})" for r in refs
        ) if isinstance(refs, list) else None)

    # Rename or fill missing columns if necessary
    if 'ErrorInformation_errorLocation_positionStart_lineNumber' not in df.columns:
        df['ErrorInformation_errorLocation_positionStart_lineNumber'] = None

    # Add Dataset/Table/Partition metadata
    df['DatasetName'] = dataset_name
    df['TableName'] = table_name
    df['PartitionName'] = partition_name

    return df


## Retrieve all semantic models/datasets and run linter on Partitions

In [ ]:


df_datasets = fabric.list_datasets(workspace_id)

# Remove rows where the 'Dataset Name' is 'Usage Metrics Report'
df_datasets = df_datasets[df_datasets['Dataset Name'] != 'Usage Metrics Report']

print(df_datasets['Dataset ID'])
from sempy_labs.tom import connect_semantic_model
import pandas as pd

results = []
for d in df_datasets.to_dict(orient="records"):
    with connect_semantic_model(dataset=d['Dataset ID'], workspace=workspace_id, readonly=True) as tom:
        for t in tom.model.Tables:
            for c in t.Partitions:
                # Skip partitions like 'LocalDateTable_...'
                if c.Name.startswith("LocalDateTable_"):
                    continue

                print(f"Linting '{t.Name}'[{c.Name}] in dataset '{d['Dataset Name']}'")

                try:
                    lint_result = invoke_code_linting(
                        code=c.Source.Expression,
                        subscription_key=subscription_key
                    )

                    df_temp = lint_result_to_df(
                        lint_result,
                        dataset_name=d['Dataset Name'],
                        table_name=t.Name,
                        partition_name=c.Name
                    )
                    # Output results
                    display(df_temp)

                    results.append(df_temp)

                except Exception as lint_err:
                    print(f"[ERROR] Linting failed for {d['Dataset Name']} -> {t.Name} -> {c.Name}: {lint_err}")
                    continue



# Combine all results into a single DataFrame
df_all = pd.concat(results, ignore_index=True)

#print(df_all)
import pandas as pd

# After creating df_all as shown previously

# Example aggregation: count of categories per TableName and Partition
# Assuming your DataFrame has columns: 'TableName', 'Partition', 'Category'

agg_df = (
    df_all
    .groupby(['DatasetName', 'TableName', 'PartitionName', 'Category'])
    .size()
    .reset_index(name='Count')
)

print(tabulate(agg_df, headers='keys', tablefmt='github', showindex=False))